In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Data Collection

In this phase we will collect the customer ratings data from the airline quality website called [Skytrax](https://www.airlinequality.com/airline-reviews/british-airways). We will collect data about airline ratings, seat ratings and lounge experience ratings from this website. 

In [2]:
#imports

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests 

In [4]:
#create an empty list to collect all reviews
reviews  = []

#create an empty list to collect rating stars
stars = []

#create an empty list to collect date
date = []

#create an empty list to collect country the reviewer is from
country = []

#create an empty list to collect the seat type the customer selected
seat_type = []

In [5]:
for i in range(1, 36):
    page = requests.get(f"https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/?sortby=post_date%3ADesc&pagesize=100")
    soup = BeautifulSoup(page.content, "html5")
    
    #reviews
    for item in soup.find_all("div", class_="text_content"):
        reviews.append(item.text)
        
    #star rating
    for item in soup.find_all("div", class_ = "rating-10"):
        try:
            stars.append(item.span.text)
        except:
            print(f"Error on page {i}")
            stars.append("None")
            
    #date
    for item in soup.find_all("time"):
        date.append(item.text)
        
    #country
    for item in soup.find_all("h3"):
        country.append(item.span.next_sibling.strip(" ()"))

    #seat type
    try:
      for table in soup.findAll("table"):
        for row in table.findAll("tr"):
          out_row = []
          for col in row.findAll("td"):
            out_row.append(col.text)
          if out_row[0] == 'Seat Type':
            seat_type.append(out_row[1])
          #if out_row[0] == 'Date Flown':
            #date.append(out_row[1])
    except requests.HTTPError as e:
      print('Unable to open url',e)

Error on page 30
Error on page 31
Error on page 31
Error on page 34
Error on page 34


In [6]:
print("reviews",len(reviews),"Stars",len(stars),"Date",len(date),"Country",len(country),"Seat Type",len(seat_type))

reviews 3500 Stars 3535 Date 3500 Country 3500 Seat Type 3498


In [7]:
min_len = min(len(reviews), len(stars), len(date), len(country),len(seat_type))
reviews = reviews[:min_len]
stars = stars[:min_len]
date = date[:min_len]
country = country[:min_len]
seat_type = seat_type[:min_len]

In [8]:
print("reviews",len(reviews),"Stars",len(stars),"Date",len(date),"Country",len(country),"Seat Type",len(seat_type))

reviews 3498 Stars 3498 Date 3498 Country 3498 Seat Type 3498


In [9]:
month =[]
year = []
for i in date:
  i = list(i.split(" "))
  month.append(i[1])
  year.append(i[2])

In [10]:
#create  a dataframe from these collected lists of data

df = pd.DataFrame({"reviews":reviews,"stars": stars,"month":month,"year":year, "country": country, "seat_type": seat_type})

In [11]:
df.head()

,reviews,stars,month,year,country,seat_type
0,"✅ Trip Verified | Very poor service, very fru...",\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,March,2023,United Kingdom,Economy Class
1,Not Verified | Generally poor. Sent to gate o...,1,March,2023,United Kingdom,Economy Class
2,Not Verified | BA changed our prepaid seats a...,3,March,2023,United Kingdom,Economy Class
3,Not Verified | Flew from London Heathrow to M...,1,March,2023,United Kingdom,Economy Class
4,Not Verified | I was meant to fly in January t...,1,March,2023,United Kingdom,Economy Class


In [12]:
df.shape

(3498, 6)

### Export the data into a csv format

In [13]:
import os

cwd = '/content/drive/MyDrive/British Airways'
df.to_csv(cwd+ "/reviews.csv")